### Python module/package imports for this chapter

In [3]:
import os, sys, collections, re, json, io, base64

In [4]:
import numpy as np

import matplotlib
import matplotlib.pyplot as pp
import matplotlib.animation as anim
from mpl_toolkits.basemap import Basemap

%matplotlib inline

In [5]:
import requests
import bs4      # BeautifulSoup 4

In [6]:
import IPython.display
import PIL, PIL.Image, PIL.ImageOps, PIL.ImageEnhance  # Python Imaging Library - now "pillow"

### Stereograph-making Python module, with code taken from previous videos

**March 2020 update**: the `mars.nasa.gov` portal has been restyled, so it now looks different than in the video. Furthermore, the website now uses JavaScript to populate the raw-image query pages dynamically, so that we cannot load those pages and find image URLs. However, the same information is available in JSON files from `api.nasa.gov`, with GET URLs similar to

    https://api.nasa.gov/mars-photos/api/v1/rovers/curiosity/photos?sol=1460&camera=FHAZ&api_key=DEMO_KEY

The code below is updated accordingly. Instead of `BeautifulSoup`, it uses `json` to parse the response. Note that the `DEMO_KEY` is limited to 30 requests per day. Those will be sufficient for what we do below, but you can generate your own key, with much larger limits, at https://api.nasa.gov.

The code will still attempt to use a local cache of json files and images, if available.

In [7]:
%%file stereo.py

import os, sys, re, io, base64, json
import requests, bs4
import PIL, PIL.Image, PIL.ImageOps, PIL.ImageEnhance

def getday(day):
    """Load the webpage collecting Curiosity Front Hazard Cam
    images for Sol day, and yield a sequence of URLs for (left,right) pairs.
    Before loading, see if the webpage is available in a local cache."""
    
    cached = os.path.join('images', str(day) + '.json')
    try:
        text = open(cached,'r').read()
    except FileNotFoundError:
        daypage = requests.get('https://api.nasa.gov/mars-photos/api/v1/rovers/curiosity/photos',
                               params={'sol': day, 'camera': 'FHAZ',
                                       'api_key': 'CeYgq3YlAUSUXMRoZJxPogOpj4a8bvmJEDv3m6G6'}) #DEMO_KEY'})
        text = daypage.text
    
    # we parse the returned JSON, which has format
    # {'photos': [{'id': XXXX, 'img_src': URL, ...},
    #             ...]
    #  ...}
    # to isolate the image URLs
    
    try:
        images = json.loads(text)
        srcs = [photo['img_src'] for photo in images['photos']]
    except:
        srcs = []
    
    print("Found {} images for day {}...".format(len(srcs),day))

    # iterate over nonoverlapping pairs in the list:
    # 0,2,4,... and 1,3,5,...
    for one, two in zip(srcs[::2],srcs[1::2]):
        # we may get the left/right in the wrong order, so check the URLs
        left, right = (one, two) if 'FLB' in one else (two, one)
        
        yield left, right

def getimage(url):
    """Load a Curiosity image from url, resize it and dewarp it.
    However, first see if the image is available in a local cache."""
    
    cached = os.path.join('images',os.path.basename(url))
    try:
        content = open(cached,'rb').read()
    except FileNotFoundError:
        content = requests.get(url).content
    
    img = PIL.Image.open(io.BytesIO(content))
    
    resized = img.resize((400,400))
    dewarped = img.transform((400,300),
                             PIL.Image.QUAD,data=(0,0,100,400,300,400,400,0),
                             resample=0,fill=1)
    
    return dewarped

def blend(left,right):
    """Colorize and blend left and right Curiosity images."""
    
    blend = PIL.Image.blend(PIL.ImageOps.colorize(left,(0,0,0),(255,0,0)),
                            PIL.ImageOps.colorize(right,(0,0,0),(0,255,255)),0.5)
    
    enhanced = PIL.ImageEnhance.Brightness(blend)
    
    return enhanced.enhance(1.75)

Overwriting stereo.py


## Making a webserver offering stereoscopic Mars images

In [8]:
%%file server.py

import flask

app = flask.Flask(__name__)

@app.route('/')
def hello_world():
    return flask.Response('<html><body><p>Hello, world!</p></body></html>')

app.run(host='0.0.0.0')

Overwriting server.py


In [9]:
%%file server.py

import flask, jinja2, io, base64

app = flask.Flask(__name__)

template = jinja2.Template("""
<html>
  <body>
    <img src="data:image/png;base64,{{imgdata}}"/>
  </body>
</html>
""")

def makeimgdata(img):
    buffer = io.BytesIO()
    img.save(buffer,format='PNG')
    return base64.b64encode(buffer.getvalue()).decode('ascii')

@app.route('/<sol>')
def getsol(sol):
    html = template(imgdata='')
    return flask.Response(html)

@app.route('/')
def hello_world():
    return flask.Response('<html><body><p>Hello, world!</p></body></html>')

app.run(host='0.0.0.0')

Overwriting server.py


**March 2020 update:** on Windows, connect to `localhost` instead of `0.0.0.0`. If that does not work with the Edge browser, you can try Firefox.

In [10]:
img = PIL.Image.open('../03_05/images/FLB_517332079EDR_F0541610FHAZ00323M_.jpg')

In [11]:
def makeimgdata(img):
    buffer = io.BytesIO()
    img.save(buffer,format='PNG')
    return base64.b64encode(buffer.getvalue()).decode('ascii')

In [12]:
b64 = makeimgdata(img)

In [13]:
len(b64)

730876

In [14]:
b64[:1000]

'iVBORw0KGgoAAAANSUhEUgAABAAAAAQACAAAAABadnRfAAEAAElEQVR4nLz9XZckW3IcipmZ+47IrOo+Z74AECBAEOQVpaX7olf9/1+iJ90liSRm5nRXZcR2Nz3syOo+Zwa8AC9ws9fqqoqMjIyP7V/m5u789wCMX7y4fvi7v83nG8TPf1v7mYRNNgiuTc+3yednCNDXJwjw4ye+bXz+Z9i2YcCG175Erz3YIqzQjthSoUGBpHAec57FohQRSpIgSRsEbNcsSDmSjHVck5CVApXueR5ntduG7W/3QSKwTgjwOh3rOh/CkGCQAEitK6UkKYK5b/uvfvv//F/XfTQBVKD1cc8rANOWacJsPm8LzbWf+fx0C4BBGJyJVkMtmA1dR0KrZX78cW3B+hUo8Xk8wNb6ShN4ntW3Deb1/K9TWF+l587ruLi2f7cDKnAdwAQMtvDc0uuM19nB+O6bYeJ5J56vFtYtMdf1WN+99+3WfHxnC9e3tSqum24a346JmWtjxXrnOvd1jwyurQ35QwSuva4faK2r8nU3P3acCYMVLQAVa/Vd3/3tyPp4wBXXDcXHPfruZT7XwfOBtM48NWse88vv/1//v9//5LN88K3f3uBpd5VtN7qNrrVm7b4kUYhx23iSfyr+a5c/2U4DS8A/dvk4vUsBXDtdm/50Ry7x5891y8cx1rfy49v83M3PD/DSBOvukyFBESQBgrDPWbYhMCiJzSWo68TsdhlKUbSBJq6PgiKo6qqyLfj6/o9zImnzqREuBcDvdOHSgd9fFnHpH0DB+O3/4x8+iQpQ7u3Yi88DWTo2z9t7ZrEBqh1FMyc1R5vUOWZg7nj0p3lGnPeTk9tJQz3m/nWfQvjkdjqLAqegGXCDYD62dvaxF7z5dERVGDGlImufQGWpzbqd6L0fGToo91al7EOji4i2YyYMNQjaMmyxhNrPGWwals/Xr8x4jIKlgrMs0yUQKDIml9Za0mNAFaVSqdk5BTuqRxn

In [29]:
%%file server.py

import flask, jinja2, io, base64
import stereo

app = flask.Flask(__name__)

template = jinja2.Template("""
<html>
  <body>
    <img src="data:image/png;base64,{{imgdata}}"/>
  </body>
</html>
""")

def makeimgdata(img):
    buffer = io.BytesIO()
    img.save(buffer,format='PNG')
    return base64.b64encode(buffer.getvalue()).decode('ascii')

@app.route('/<sol>')
def getsol(sol):
    left, right = stereo.getday(sol).__next__()
    img = stereo.blend(stereo.getimage(left),stereo.getimage(right))
    
    html = template.render(imgdata=makeimgdata(img))
    
    return flask.Response(html)

@app.route('/')
def hello_world():
    return flask.Response('<html><body><p>Hello, world!</p></body></html>')

app.run(host='0.0.0.0')

Overwriting server.py


In [34]:
import stereo
# help(stereo)
n = 1333
left, right = stereo.getday(n).__next__()
print(type(left), type(right))
l, r = stereo.getimage(left),stereo.getimage(right)
fig = plt.figure(figsize=(10,7))
ax = fig.add_subplot(2,1, 1)
pp.imshow(l)
fig.add_subplot(1,2,2)
pp.imshow(r)
#print(stereo.getday(n).__next__())

Found 2 images for day 1333...
<class 'str'> <class 'str'>


KeyboardInterrupt: 

In [33]:
!ffmpeg

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with Apple clang version 11.0.0 (clang-1100.0.33.17)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.2.2_2 --enable-shared --enable-pthreads --enable-version3 --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libmp3lame --enable-libopus --enable-librubberband --enable-libsnappy --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librtmp --enable-libspeex --enable-libsoxr --enable-videotoolbox --disable-libjack --disable-indev=jack
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat   

In [28]:
!python3 server.py

 * Serving Flask app "server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
Found 2 images for day 1333...
Found 2 images for day 1333...
Found 2 images for day 1333...
[2020-06-09 06:15:05,948] ERROR in app: Exception on /1333 [GET]
Traceback (most recent call last):
  File "/Users/prajjwaldangal/summer2020/efficient_python/Ex_Files_Python_Efficiently/Exercise Files/chapter3/03_07/stereo.py", line 48, in getimage
    content = open(cached,'rb').read()
FileNotFoundError: [Errno 2] No such file or directory: 'images/FLB_515827474EDR_F0540938FHAZ00323M_.JPG'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/urllib3/connection.py", line 157, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw
  File "/opt/anaconda3/lib/python3.7/site-packages/urllib3/uti

[2020-06-09 08:51:21,629] ERROR in app: Exception on /1333 [GET]
Traceback (most recent call last):
  File "/Users/prajjwaldangal/summer2020/efficient_python/Ex_Files_Python_Efficiently/Exercise Files/chapter3/03_07/stereo.py", line 48, in getimage
    content = open(cached,'rb').read()
FileNotFoundError: [Errno 2] No such file or directory: 'images/FLB_515827474EDR_F0540938FHAZ00323M_.JPG'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/urllib3/connection.py", line 157, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw
  File "/opt/anaconda3/lib/python3.7/site-packages/urllib3/util/connection.py", line 61, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/opt/anaconda3/lib/python3.7/socket.py", line 752, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gai

127.0.0.1 - - [09/Jun/2020 08:51:49] "GET / HTTP/1.1" 200 -
^C


In [35]:
open('locations.xml', 'r').readlines()[:19]

FileNotFoundError: [Errno 2] No such file or directory: 'locations.xml'